In [4]:
### Importing required python package
import pandas as pd
import numpy as np
import json          
from pyspark.sql import SparkSession 

### This code is for POC purpose. Dont deploy this in production environment without tuning further and 
### implement exception handling

In [5]:
import time
starttime = time.time()

from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2018-07-27 09:28:04'

In [6]:
## Define Increment Load or Full Load
## Precautions for full load. Manually drop spark_pax_flight_final hive table before running this process.
loadtype = 'incrementalload'
#loadtype = 'fullload'
#RUN_DATE = '20180620000000' #today - 6 days
RUN_DATE = '20180731000000' #today - 6 days

In [7]:
import datetime

# naive datetime
rundate = datetime.datetime.strptime(RUN_DATE, '%Y%m%d%H%M%S')

print("The entire load will happen upto {}".format(rundate))
print("and this is {}".format(loadtype))


The entire load will happen upto 2018-05-31 00:00:00
and this is incrementalload

In [8]:
### Create spark session 
### This steps required to run from jupyter
spark = SparkSession.builder.appName("First SparkSession").getOrCreate()

In [9]:
sqlContext.sql(" CREATE TABLE IF NOT EXISTS loadtable ( rundate string ) ")

DataFrame[]

In [10]:
sqlContext.sql("INSERT INTO TABLE loadtable SELECT {0} ".format(RUN_DATE))

DataFrame[]

In [ ]:
### Bringing the dataset for flights
spark_flight = spark.sql("select id, airline_designator, flight_number, CONCAT(from_unixtime(UNIX_TIMESTAMP(operation_date, 'yyyy-MM-dd'), 'yyyyMMdd' ), operation_time) as flight_boarding_time , board_point, aircraft_type, class_configuration, pantry_code, crew_code, gate_number, baggage_concept, flight_status from macs_flight_mvp3 union all select  id, airline_designator, flight_number, CONCAT(from_unixtime(UNIX_TIMESTAMP(operation_date, 'yyyy-MM-dd'), 'yyyyMMdd' ), operation_time) as flight_boarding_time , board_point, aircraft_type, class_configuration, pantry_code, crew_code, gate_number, baggage_concept, flight_status from h_macs_flight_full_new ")

In [ ]:
# spark_flight.printSchema()

In [ ]:
### testing spark_flight data 
# print(spark_flight.count()) ### Testing result 1884964 as of 6/19
#spark_flight.show()

In [ ]:
## spark_flight_new = spark_flight.select("flight_number" , "flight_boarding_time" , "board_point" ).unique()

spark_flight_new = spark_flight.drop_duplicates([ "flight_number" , "flight_boarding_time" , "board_point"] )

In [ ]:
### testing spark_flight data 
# print(spark_flight_new.count()) ### Testing result 1884964 as of 6/19  8936516
# spark_flight.show()
# type(spark_flight_new)

In [ ]:
### Bringing the dataset for pax
spark_pax = spark.sql("select pax_id, pax_booking_status, destination, cabin_class, transit_indicator, booked_class, travelled_class, seat_number as SeatNumber, pax_type, pax_group_code, boarding_date, upgrade_indicator, ticket_class, handicapped, unaccompanied_minor, priority_pax, check_in_city, date_of_birth, nationality, gender, country_of_birth, country_of_residence, mfl_id as id from h_macs_pax_joined_all_new where boarding_date >= '2016-12-12 00:00:00.0' and upper(travelled_class) = 'J' UNION select pax_id, pax_booking_status, destination, cabin_class, transit_indicator, booked_class, travelled_class, seat_number as SeatNumber, pax_type, pax_group_code, boarding_date, upgrade_indicator, ticket_class, handicapped, unaccompanied_minor, priority_pax, check_in_city, date_of_birth, nationality, gender, country_of_birth, country_of_residence, mfl_id as id from macs_pax_joined_mvp3 where ( cabin_class = 'J' or travelled_class = 'J' ) ") 

# and operation_date > ( select from_unixtime(unix_timestamp(max(rundate),'yyyyMMddHHmmss' )) from loadtable ) ")


In [ ]:
### testing spark_pax data 
# print(spark_pax.count())  ## 20667814  8936516
# spark_pax.show()

In [ ]:
# from pyspark.sql.functions import broadcast

spark_pax_flight_key  = [ "id" ]
spark_pax_flight = spark_flight_new.join(spark_pax , spark_pax_flight_key , "inner")

In [ ]:
### testing spark_pax_flight data 
### print(spark_pax_flight.count())
#spark_pax_flight.show()
print(loadtype)

In [ ]:
### Loading the pax and flights data into Hive table
spark_pax_flight.createOrReplaceTempView("spark_pax_flight")

if loadtype is "fullload" :
    print("loading spark_pax_flight_final table")
    sqlContext.sql("create table spark_pax_flight_final as select * from spark_pax_flight")
else:
    print("Dropping spark_pax_flight_final table")
    sqlContext.sql("drop table IF EXISTS spark_pax_flight_final")
    print("loading spark_pax_flight_final table")
    sqlContext.sql("create table spark_pax_flight_final as select * from spark_pax_flight")    

In [ ]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

In [ ]:
endtime = time.time()
diff = int ( endtime - starttime ) 
minutes, seconds = diff // 60, diff % 60

print("time taken: {} mins & {} secs ".format(minutes,seconds))